In [ ]:
import _init_paths
import tensorflow as tf

#import matplotlib
#matplotlib.use('Agg') # Must be before importing matplotlib.pyplot or pylab!
import matplotlib.pyplot as plt
%matplotlib inline

from fast_rcnn.config import cfg
from fast_rcnn.test import im_detect
from fast_rcnn.nms_wrapper import nms
from utils.timer import Timer
import numpy as np
import os, sys, cv2
import argparse
from networks.factory import get_network

CLASSES =('__background__', 'arm')

# CLASSES =('__background__', 
#                             "box",
#                             "gum",
#                             "marker",
#                             "pen",
#                             "postit",
#                             "scissors",
#                             "tape",
#                             "usb"
#                              )


In [ ]:

#CLASSES = ('__background__','person','bike','motorbike','car','bus')

def vis_detections(im, class_name, dets,ax, thresh=0.5):
    """Draw detected bounding boxes."""
    inds = np.where(dets[:, -1] >= thresh)[0]
    if len(inds) == 0:
        return

    for i in inds:
        bbox = dets[i, :4]
        score = dets[i, -1]

           ax.text(bbox[0], bbox[1] - 2,
                '{:s} {:.3f}'.format(class_name, score),
                bbox=dict(facecolor='blue', alpha=0.5),
                fontsize=14, color='white')

    ax.set_title(('{} detections with '
                  'p({} | box) >= {:.1f}').format(class_name, class_name,
                                                  thresh),
                  fontsize=14)
    plt.axis('off')
    plt.tight_layout()
    plt.draw()


In [ ]:

def demo(sess, net, image_name):
    """Detect object classes in an image using pre-computed object proposals."""

    # Load the demo image
    im_file = os.path.join(cfg.DATA_DIR, 'office_supplies', 'images', image_name)
    #im_file = os.path.join('/home/corgi/Lab/label/pos_frame/ACCV/training/000001/',image_name)
    im = cv2.imread(im_file)

    # Detect all object classes and regress object bounds
    timer = Timer()
    timer.tic()
    scores, boxes = im_detect(sess, net, im)
    timer.toc()
    print ('Detection took {:.3f}s for '
           '{:d} object proposals').format(timer.total_time, boxes.shape[0])

    # Visualize detections for each class
    im = im[:, :, (2, 1, 0)]
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.imshow(im, aspect='equal')

    CONF_THRESH = 0.15
    NMS_THRESH = 0.08
    for cls_ind, cls in enumerate(CLASSES[1:]):
        cls_ind += 1 # because we skipped background
        cls_boxes = boxes[:, 4*cls_ind:4*(cls_ind + 1)]
        cls_scores = scores[:, cls_ind]
        dets = np.hstack((cls_boxes,
                          cls_scores[:, np.newaxis])).astype(np.float32)
        keep = nms(dets, NMS_THRESH)
        dets = dets[keep, :]
        vis_detections(im, cls, dets, ax, thresh=CONF_THRESH)


In [ ]:
cfg.TEST.HAS_RPN = True  # Use RPN for proposals
gpu_id = 1
demo_net = "VGGnet_test"
# model = "/data/code/Faster-RCNN_TF/output/faster_rcnn_end2end/office_supplies/VGGnet_fast_rcnn_iter_200.ckpt"
model = "/data/code/Faster-RCNN_TF/output/faster_rcnn_end2end/armpos/VGGnet_fast_rcnn_iter_2000.ckpt"

In [ ]:
cfg.DATA_DIR

In [ ]:
# image_name = 'pedestrian_cars.jpg'
# im_file = os.path.join(cfg.DATA_DIR, 'demo', image_name)
im = cv2.imread("/data/code/Faster-RCNN_TF/data/armpos/images/10.jpg")
im.shape

In [ ]:
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))


In [ ]:
net = get_network(demo_net)

In [ ]:
saver = tf.train.Saver()

In [ ]:
saver.restore(sess, model)

print('\n\nLoaded network {:s}'.format(model))

In [ ]:
with tf.variable_scope('cls_score', reuse=True):
    print(tf.get_variable('weights').get_shape())
    print(tf.get_variable('biases').get_shape())

In [ ]:
layers = net.layers
# out_layers = {'cls_score', 'bbox_pred', 'roi-data'}
out_layers = {}
out_dict = {}
for layer in layers:
    if layer in out_layers:
        continue
    allvsrs = tf.get_collection(tf.GraphKeys.VARIABLES, layer)
    if len(allvsrs) > 0:
        d={}
        for variable in allvsrs:
            d[variable.name.split('/')[1].split(':')[0]] = variable.eval(session=sess)
        out_dict[layer] = d
# with tf.variable_scope(, reuse=True):
#     td = tf.get_variable('weights')


In [ ]:
splits = ['obj', 'arm']
excepts = {'conv1_1','conv1_2', 'conv2_1', 'conv1_1'}
def split_model(model_dict, splits, excepts):
    split_dict = {}
    split_maps = [(key, [split + '_' + key for split in splits]) 
     if key not in excepts 
     else (key, [key])  for key in model_dict.keys()]
    for (inky, outkys) in split_maps:
        for outky in outkys:
            split_dict[outky] = model_dict[inky]
    return split_dict
    

In [ ]:
split_dict = split_model(out_dict, splits, excepts)

In [ ]:
split_dict.keys()

In [ ]:
with open('/data/tmp/split_trained_model.npy', 'w') as f:
    np.save(f, split_dict)

In [ ]:
split_dict['arm_fc7']

In [ ]:

im_names = ['11.jpg', '33.jpg', '44.jpg', '55.jpg', '66.jpg']
#im_names = ['000456.jpg']
for im_name in im_names:
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print('Demo for data/demo/{}'.format(im_name))
    demo(sess, net, im_name)

plt.show()

In [ ]:
filename = '/data/robotics/raw_position_images/train_2' + '.tfrecords'

example = tf.train.Example()
pi = tf.python_io.tf_record_iterator(filename)
example.ParseFromString(pi.next())

dat = np.fromstring(example.features.feature["left_image"].bytes_list.value[0], dtype=np.uint8)


im = dat.reshape((480, 640, 3))


In [ ]:
list(enumerate([1,2][1:]))

In [ ]:
scores, boxes = im_detect(sess, net, im)

In [ ]:
scores.shape

In [ ]:
# im = im[:, :, (2, 1, 0)]
fig, ax = plt.subplots(figsize=(12, 12))
ax.imshow(im, aspect='equal')

CONF_THRESH = 0.04
NMS_THRESH = 0.03
for cls_ind, cls in enumerate(CLASSES[1:]):
    cls_ind += 1 # because we skipped background
    print cls_ind
    print scores.shape
    cls_boxes = boxes[:, 4*cls_ind:4*(cls_ind + 1)]
    cls_scores = scores[:, cls_ind]
    dets = np.hstack((cls_boxes,
                      cls_scores[:, np.newaxis])).astype(np.float32)
    keep = nms(dets, NMS_THRESH)
    dets = dets[keep, :]
    print(keep)
    vis_detections(im, cls, dets, ax, thresh=CONF_THRESH)


In [ ]:
dets = np.hstack((cls_boxes,
                  cls_scores[:, np.newaxis])).astype(np.float32)
keep = nms(dets, NMS_THRESH)


In [ ]:
np.where(dets[:, -1] >= 0.04)[0]

In [ ]:
dets[keep, -1]

In [ ]:
cls_ind = 2
boxes[:, 4*cls_ind:4*(cls_ind + 1)]

In [ ]:

fig, ax = plt.subplots(figsize=(12, 12))
ax.imshow(im)
fig.canvas.mpl_connect( "button_press_event", )

In [ ]:
scores, boxes = im_detect(sess, net, im)

In [ ]:
len(scores), len(boxes)

In [ ]:
np.hstack([scores, boxes]).

In [ ]:
import json
json.dumps(d)

In [ ]:
(x0, y0, x1, y_1) = linebuilder.x_0, linebuilder.y_0, linebuilder.x_press, linebuilder.y_press
